# Periodic Boundary Conditions - Structure 

## Overview

In this session, we will learn how to compute structural quantities from an MD simulation. 

### Questions

- What are PBCs? What is the minimum image convention? What are images?
- How do I assess the structure of a liquid?
- How do I compute structural quantities of polymers from a MD simulation?

### Objectives

- Write a function that computes the correct distance in PBC.
- Calculate the radial distribution function of a system. 
- Calculate the radius of gyration of a polymer.
- Show that the simple polymer model agrees with scaling expectations (Rouse model).

## Boilerplate code

In [ ]:
import matplotlib
import numpy
import scipy.stats
import gsd, gsd.hoomd
import freud
import math
import itertools
import hoomd
# for movie/renders
import fresnel
import PIL
import io
import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

%matplotlib inline
matplotlib.style.use("ggplot")
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

def render(frame):
    scene = fresnel.Scene()
    geometry = fresnel.geometry.Sphere(scene, N=frame.particles.N)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]), roughness=0.5)
    geometry.position[:] = frame.particles.position
    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(scene, frame.configuration.box,box_radius=0.01)
    L = frame.configuration.box[0]
    scene.camera = fresnel.camera.Perspective(position=(L*1.8, L*1.8, L * 2.2), look_at=(0, 0, 0), up=(0, 1, 0), height=0.28)

    scene.background_color =(1,1,1)
    scene.background_alpha = 1
    if frame.bonds.N>0:
        geometry.radius[:] = [0.2]*frame.particles.N

        all_bonds = numpy.stack(
        [
            frame.particles.position[frame.bonds.group[:, 0]],
            frame.particles.position[frame.bonds.group[:, 1]],
        ],
        axis=1,
        )
        # Use a distance cutoff (L/2) to filter bonds that span the periodic boundary
        bond_distances = numpy.linalg.norm(all_bonds[:,0,:]-all_bonds[:,1,:], axis=1)
        L = frame.configuration.box[0]
        bond_indices = numpy.where(bond_distances < L/2)[0]
        filtered_bonds = all_bonds[bond_indices, :, :]

        bonds = fresnel.geometry.Cylinder(scene, N=len(filtered_bonds))
        bonds.material = fresnel.material.Material(roughness=0.5)
        bonds.outline_width = 0.05

        bonds.points[:] = filtered_bonds
        bonds.radius[:] = [0.1]*len(filtered_bonds)
        bonds.material.primitive_color_mix = 1.0
        bonds.color[:] = fresnel.color.linear([0.8, 0.8, 0.8])

    return fresnel.preview(scene)

# Boundary Conditions - Structure 

## Minimum Image Convention 

Write a function `dist(..)` that computes the distances between points in 3D without periodic boundary conditions. Then write one that does take periodic boundary conditions into account, `dist_pbc(..)`. Your functions should work for arrays of positions (which might contain one or more points) and return an array of distances. 

*Hint:* feel free to use numpy (`numpy.where`, `numpy.linalg.norm`, `numpy.sqrt`, `numpy.abs`,..) functions.

We assume **hoomd** conventions, meaning the box coordinates are defined from `(-Li/2,+Li/2)` (with `i` = x,y, or z) and the origin of the box is at `(0,0,0)`.

In [ ]:
def dist(x1,x0):
    # add code
    return ...

def dist_pbc(x1,x0,Box):
    # add code
    return ...

Test your functions with points in 3D:

In [ ]:
Box = numpy.array([10,10,10])

points_A = numpy.array([[2,3,1]])
points_B = numpy.array([[0,0,0],[-1,-4,0],[-4,2,3],[3,5,-4]])
points_C = numpy.array([[3,0,-2],[2,-2,4],[2,0,0],[0,0,5]])
points_D = numpy.array([[-1,0,0]])

print(dist_pbc(points_A,points_B,Box))
print(dist(points_A,points_B))

print(dist_pbc(points_B,points_C,Box))
print(dist(points_B,points_C))

print(dist_pbc(points_A,points_A,Box))
print(dist_pbc([points_B[0]],points_D,Box))


*Hint:* The expected output for the above should be: 
```
[3.74165739 4.35889894 4.58257569 5.47722558]
[3.74165739 7.68114575 6.40312424 5.47722558]
[3.60555128 5.38516481 5.38516481 5.91607978]
[ 3.60555128  5.38516481  7.         10.72380529]
[0.]
[1.]
```
Only continue if your `dist_pbc` function is correct! 

### Additions 

Try to come up with alternative ways to calculate the minimum distance: 
1. use `if` statements
2. use `floor/round`
3. use other `numpy` functions
4. ....

## Radial Distribution Function

Now we want to compute the radial distribution function. For this we will use the minium distance functions from above, so make sure they are correct before proceeding! 

It is useful to use `numpy.histogram` for this task, since we effectively are computing a radial histogram. Normalized correctly, this will give us the radial distribution function. 

First, write a function that returns the correct normalization for an array of distances `r`, as function of `N` and the `Box`.  

*Hint:* Think about what the number of particles in a spherical shell would be. What is its volume? You can find the correct normalizations [here](https://en.wikibooks.org/wiki/Molecular_Simulation/Radial_Distribution_Functions). 

In [ ]:
def norm(Box,N,r):
    # add code
    ...
    return norm

Now write a function that takes two arrays `A`,`B` of points, a `Box`, number of bins `number_of_bins`, and a maximum distance `rmax` and returns the pair correlation function as a histogram and the corresponding bin centers. Set up an empty histogram with the correct number of bins, that you can then add to.

In [ ]:
def pair_corr(A,B,Box,number_of_bins,rmax):
    # add a definition of the bins. Use numpy.linspace
    bins = ...
    hist_all, bin_edges_all = numpy.histogram(0,bins)

    for particle in A:
        # compute distances between current particle from A and all particles in B
        # generate a histogram, add it to the global one
        d = ...
        # bin all the distances by making a histogram out of them (with exactly the same bins as the histogram "hist_all")
        hist, bin_eges = numpy.histogram(...)
        # then we can add the histograms together to keep track of the cummulative result:
        hist_all += hist

    # compute the bin centers from the bin edges.
    bincenters = ..
    # normalize histogram
    # set first entry to 0 (Why?)
    ...
    return bincenters,hist_all

## Short LJ Simulation 

Now we need to test our code. For this, we will run a short LJ simulation. For simplicity, we use a simple cubic as starting configuration. let's put some particles on a simple cubic lattice into a box. Define the starting `Frame`: 

In [ ]:

frame = gsd.hoomd.Frame()

L = 12
spacing = 1.05
x = numpy.linspace(-0.5*L, 0.5*L, int(L/spacing), endpoint=False)
position = list(itertools.product(x, repeat=3))

frame.particles.N = ...
frame.particles.position = ...
frame.configuration.box = [L, L, L, 0, 0, 0]
frame.particles.types = ['A']


Render the frame and check that it is correct:

In [ ]:
render(frame)

Let's compute the radial distribution function of that simple cubic configuration and plot $g(r)$ as function of $r$.

In [ ]:
box = ...
r,gr = pair_corr(...)
fig, ax = matplotlib.pyplot.subplots()
ax.plot(...)
matplotlib.pyplot.show()

What is the longest distance you can reasonably compute the radial distribution function for and why?

### Run the Simulation 

Now we can run a short simulation at temperature $1\,k_BT$, which should melt the simple cubic into a liquid. Look up how to [save a trajectory](https://hoomd-blue.readthedocs.io/en/v4.7.0/module-hoomd-write.html#hoomd.write.GSD) into a `gsd` file. 

In [ ]:
integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer=0.4)

# define LJ interactions and attach them to the simulation
# Hint: you have done this yesterday!
LJ = ...

simulation = hoomd.Simulation(device=hoomd.device.auto_select(),seed=8)
simulation.operations.integrator = integrator
simulation.create_state_from_snapshot(frame)

nvt = hoomd.md.methods.Langevin(filter=hoomd.filter.All(),kT=1)
integrator.methods.append(nvt)
simulation.operations.integrator = integrator

gsd_out = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(100),
    mode='wb',
    filename='example_lj.gsd')

simulation.operations.writers.append(gsd_out)

simulation.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=1)
simulation.run(10000)
gsd_out.flush()

Render the last snapshot of the simulation:

In [ ]:
frame = simulation.state.get_snapshot()
render(frame)

Now we can measure the $g(r)$ of the liquid by taking the end of the trajecory and averaging the radial distribution function of those snapshots. Look up how to [open and loop](https://gsd.readthedocs.io/en/stable/hoomd-examples.html#hoomd-examples) over a `gsd` trajectory. 

In [ ]:
trajectory =  gsd.hoomd.open(name=..., mode='r')
# from the first frame of the trajectory (trajectory[0].configuration.box) take the first three elements to define the box size:
box = ...
# this list will be used to accumulate our results
gr = []
# take just last 30 snapshots
for frame in trajectory[-30:]:
    # read the particle positions from the frame
    pos = ...
    # calculate the pair correlation function for this frame:
    r,hist = pair_corr(...)

    gr.append(hist)

Now we need average the array of all results per frame over all frames to plot the results:

In [ ]:
fig, ax = matplotlib.pyplot.subplots()
# add code
# average the histograms you collected above. What axis do you need to average over?
hist_averaged = numpy.average(...)
# Hint: this is the same as your plot above, except that now we are plotting an accumulated g(r) over the last 30 frames of a trajectory
ax.plot(...)
# add axis labels etc.
matplotlib.pyplot.show()

### Freud - Radial Distribution Function

It is always advisable to know exactly what you analyze and calculate from a simulation, so the exercise above lets us compute $g(r)$ and check our code and understanding. However, often, this might be quite slow and eventually cumbersome. There are many packages that allow us to compute certain standard properties from simulations without needing to implement everything ourselves. [**Freud**](https://freud.readthedocs.io/) is such a package. Check the [freud documentation on $g(r)$](https://freud.readthedocs.io/en/latest/modules/density.html#freud.density.RDF). You might find the tutorial on the [radial distribution function](https://freud.readthedocs.io/en/latest/gettingstarted/examples/module_intros/density.RDF-AccumulateFluid.html) helpful as well.

With this knowledge, write code that computes $g(r)$ using freud: 

In [ ]:
rdf = freud.density.RDF(bins=..., r_max=...)
for frame in trajectory[-30:]:
    # define the box (this time the whole 6 numbers from the frame.configuration.box and not just the first three)
    box = ...
    # particle positions of all particles in the frame:
    points =...
    rdf.compute(system=(box, points), reset=False)

### Compare

Let's now compare our version with the freud version by plotting both of them: 

In [ ]:
fig, ax = matplotlib.pyplot.subplots()

ax.plot(rdf.bin_centers, rdf.rdf)
ax.plot(r,hist_averaged,linestyle='dashed')
ax.set_xlabel('r')
ax.set_ylabel('g(r)')
matplotlib.pyplot.show()

### Additions 

1. Generate random uniform positions to sample configurations of an ideal gas. What is the $g(r)$ of an ideal gas? What is your expectation for the outcome and why? Can we use this to test our code?
2. Show how the $g(r)$ changes as function of temperature and/or density of the system.
3. What is the $g(r)$ of a perfect fcc crystal? *Hint:* You can use a similar setup to above to generate a fcc crystal. 

## Radius of Gyration of a Polymer

Now we want to calculate some structural quantities of polymers. We will use the radius of gyration (see [Rubinstein & Colby, Chapter 2.4](https://www.eng.uc.edu/~beaucag/Classes/Properties/Books/M.%20Rubinstein,%20Ralph%20H.%20Colby,%20-%20Polymer%20Physics%20(Chemistry)%20(2003).pdf)) $R_g$ as an example. 

To save some time, we will reach into the future and pretend we have run the simulations already. We will run some of them in the next session! For now, let's open a trajectory of a Gaussian chain model:  

In [ ]:
trajectory = gsd.hoomd.open(name='../08-boundaries-periodic-2/run_len_80_pol_10.gsd', mode='r')

Let's render a movie of the trajectory to visualize what we are working with. The trajectory seems to be pretty long and has 10 polymers of length 80 in it. 

In [ ]:
ims = []

a = render(trajectory[0])
im0 = PIL.Image.fromarray(a[:, :, 0:3], mode="RGB").convert("P", palette=PIL.Image.Palette.ADAPTIVE)
for frame in trajectory[1::500]: # only take very 500th frame
    a = render(frame)
    im = PIL.Image.fromarray(a[:, :, 0:3], mode="RGB").convert("P", palette=PIL.Image.Palette.ADAPTIVE)
    ims.append(im)

filename = 'test.gif'
im0.save(filename, "gif", save_all=True, append_images=ims, duration=20, loop=0)

with open(filename,'rb') as f:
    display.Image(data=f.read(), format='png')


Now we can compute the radius of gyration. For this, we do need to first unwrap the positions into an array called `unwrapped_positions`. Then, the positions need to be split (`numpy.split`) into seperate arrays, one per polymer. For this determine where the `polymer_delimiters` are, i.e. the indices of the breaks between the polymers.  Then we need to determine the center of mass of each polymer (`com`), and then calculate the $Rg^2$ of each polymer. Once the entire trajectory is processed, we can take the square root to get $R_g$ later. 

Hints: 
1. For unwrapping the coordinates, you need to add `frame.particles.image*frame.configuration.box[0:3]` to the positions, since the `images` are integers keeping track of the number of image crossings. 
2. To compute the `polymer_delimiters` you can make a list manually, that holds the indices of the ends, for example for polymers of $N=80$, this list would be `[ 80 160 240 320 400 480 560 640 720]`, because the first polymer goes from index `[0-79]`, the next one from `[80-159]` etc. Of course you do not want to compute this manually, because then you would have to change this code every time you cange the chain length, so you need to find a way to do this automatically. One way to do this for linear polymer chains, is to look for "breaks" in the bond array. The bond array contains `[[0,1],[1,2],...,[78 79],[80 81],...]` where you can see that except for at the *end* of a polymer, the first index of one bond is identical to the second index of the previous bond. So if we use `numpy.where`, we can compute where the difference of  `bonds[:-1,1]-bonds[1:,0]` is NOT zero. 
3. for computing the center of mass `com`, you can use `numpy.mean`
4. for comuting the $R_g^2$, radius of gyration squared, you can use `numpy.average` of the `numpy.sum`. What do you need to sum? Which axis do you need to sum over? 
5. 

In [ ]:
rg2 = []
timestep = []
for frame in trajectory:
    unwrapped_positions = ...
    bonds = ...
    polymer_delimiters = bonds[...]
    polymers  = numpy.split(...)
    for p in polymers:
        com = numpy.mean(p,axis=...)
        Rg2 = numpy.average(numpy.sum(..., axis=...))
        rg2.append(Rg2)
        timestep.append(frame.configuration.step)

Plot the results as function of time. What are the units of $R^2_g$? What are the units of time? 

In [ ]:
fig, ax = matplotlib.pyplot.subplots()

line = ax.plot(timestep,rg2)
xlabel = ax.set_xlabel('timestep')
ylabel = ax.set_ylabel('R_g^2')
matplotlib.pyplot.show()

Note that if you did ignore PBC (didn't apply minimum image convention, or didn't write out images, or didn't unwrap correctly etc.) then one could potentially see this in this time series as large spikes of unreasonably large values. 

It is also useful to plot time series to check if a system has equilibrated or if quantities are still changing. Here, we should be ok. 

From this times series, we can then compute the average $\langle R^2_g \rangle$, and then finally $\langle R_g \rangle$:

In [ ]:
av_Rg2 = numpy.mean(...)
av_Rg = numpy.sqrt(...)

print("Rg=%1.2f"%(av_Rg))

The folder contains trajectories for $N=10,20,30,...100$. Determine their $R_g$ as function of *N*. **What scaling law do they follow?** *Hint:* Check Runbinstein and Colby for the appropriate formulas.

In [ ]:
Ns = numpy.array([10,20,30,40,50,60,70,80,90,100])
av_Rg = []
for N in Ns:
    # add code to compute Rg for each -it is the same as above.
    # Alternatively, you can also write a function that computes Rg instead of copying the code to here
    trajectory = gsd.hoomd.open(name='../8-pbc-dynamics/run_len_%s_pol_10.gsd'%N, mode='r')
    rg2 = []
    for frame in trajectory[:-500]:
        unwrapped_positions = ...
        bonds =...
        polymer_delimiters = ...
        polymers  = ...
        for p in polymers:
            com = ...
            dist_pol_com = ...
            Rg2 = ...
            rg2.append(Rg2)
    av_Rg.append(numpy.sqrt(numpy.mean(rg2)))

fig, ax = matplotlib.pyplot.subplots()

# plot results and scaling here
points = ax.scatter(Ns,av_Rg)
line = ax.plot(Ns,(Ns/6)**(1/2),c='blue')
xlabel = ax.set_xlabel(r'$N$')
ylabel = ax.set_ylabel(r'$\langle R_g\rangle$')
matplotlib.pyplot.show()

### Additions 

1. The polymers are obviously "ideal" polymers, since they follow the scaling for ideal polymers perfectly. What would we have to do to make them "non-ideal" and change the effective solvent quality they are in?
2. What scaling laws do you expect polymers in different solvent qualities to follow? 

### More Additions

1. There is an alternative way to compute the COM that does not involve unwrapping with `images`. See [J. Chem. Phys. 162, 204103 (2025)]( https://doi.org/10.1063/5.0260928). Implement this algorithm.  